In [4]:
import torch
import torch.nn as nn
import numpy as np

In [6]:
class SteeringMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super().__init__()
        layers = []
        d = input_dim

        # define layers
        for h in hidden_dims:
            layers.append(nn.Linear(d, h))
            layers.append(nn.ReLU())
            d = h
        layers.append(nn.Linear(d, 1))

        # create sequential model
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        # forward pass
        return self.net()

In [11]:
import os
import pandas as pd
def load_dataset(dataset_path):
    X = []
    y = []
    for file in os.listdir(dataset_path)[:5]:

        file_path = os.path.join(dataset_path, file)
        df = pd.read_csv(file_path)
        # Exclude 'steer' and 't' and 'steerFiltered' for X, keep all other columns
        X_cols = [col for col in df.columns if col not in ('steer', 't', 'steerFiltered')]
        X_vals = df[X_cols].values
        y_vals = df['steerFiltered'].values
        X.append(X_vals)
        y.append(y_vals)

    print(len(y)) 
    return X, y 

load_dataset("/Users/Toni/Vault/controls_challenge/data/FORD_MAVERICK_1ST_GEN")

5


([array([[1.511980414390564, 0.2039114385843277, 2.299999952316284, ...,
          -0.0012378050418238, -0.0002111188837033,
          "b'NZ6C-14D003-AL\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'"],
         [1.5358623997739451, 0.2274352542120628, 2.1993746221369497, ...,
          -0.0013110374121447, -0.0006031951972211,
          "b'NZ6C-14D003-AL\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'"],
         [1.5545496611092209, 0.2053799685437462, 2.100002615504777, ...,
          -0.0010806296816502, -0.0010040060216133,
          "b'NZ6C-14D003-AL\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'"],
         ...,
         [2.440439622687838, 0.6644743712672438, 3.599999904632569, ...,
          -0.0072747194366496, -0.003432074690977,
          "b'NZ6C-14D003-AL\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'"],
         [2.5136156222100925, 0.6946276980465448, 3.5999999046325684, ...,
          -0.0061617947030684, -0.0028405758274207,
          "b'NZ6C-14D003-AL\\

In [ ]:
def train_model(model, dataset, epochs, batch_size, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    xs, ys = load_dataset("placeholder.csv")

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0

        for x, y in xs, ys:
            x = x.to(device)
            y = y.to(device)

            pred = model(x)
            loss = loss(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * x.size(0)

        avg_loss = total_loss / len(dataset)

    torch.save(model.state_dict(), "steering_mlp.pt")
    return model